I will be using a pre-trained model created for this challenge. Creating this generic model helped me understand the implementation and reasoning behind FCN architecture, specifically how the upsampling paths function. One area of these models I did not fully understand prior to this implementation was the effect that the large stride needed in upsampling for FCN-32 and FCN-16 has on final matting map coarseness. 

I have chosen FCN-8 for two main reasons. The first is that the CelebA dataset has a large number of input dimensions and aspect ratios in the image set. Resizing and padding could be used to get all images to a standard size, but that would lose some of the finer details at boundaries in the images. In particular, older segmentation models seem to struggle with small-pixel features such as single strands of hair. This architecture has output match input image size, though I would pair this with a batch generator to run same-size images concurrently. The second reason I chose FCN-8 was that human portraits are generally posed in very similar states. Relative locations within the image are important. For example, pixels that are above hair but do not match hair characteristics are likely background. The lack of dense layers and the addition of skips from early pooling steps help to preserve spatial information. 

In [1]:
#Model creation will start by taking layers from VGG-16
def load_vgg_16(vgg_model_path):
    model = tf.saved_model.loader.load(['VGG16'], vgg_model_path)
    
    graph = tf.get_default_graph()
    image_in = graph.get_tensor_by_name('image_input:0')
    keep_per = graph.get_tensor_by_name('keep_prob:0')
    
    #3,4,7 are specifically needed as they are used to create skip-layers used in the upsampling process
    #L_7 is the last stage before this model deviates from VGG-16 architecture
    L_3 = graph.get_tensor_by_name('layer3_out:0')
    L_4 = graph.get_tensor_by_name('layer4_out:0')
    L_7 = graph.get_tensor_by_name('layer7_out:0')
    
    return image_in, keep_per, L_3, L_4, L_7

In [ ]:
#Now we need to create the remaining layers.
#When creating this, I was envisioning only segmenting the foreground and background. Therefore, I would only use 2 classes.
def layer_create(L_3_out, L_4_out, L_7_out, num_classes = 2):

    # Would be the first dense layer in VGG architecture. 1x1 convolution for FCN-8. Happily, i get to name my first layer fcn8
    fcn_8 = tf.layers.conv2d(L_7_out, filters=num_classes, kernel_size=1, name="fcn_8")

    # Upsampling to match layer 4. FCN-8 skip architecture connects this layer to L_4
    fcn_9 = tf.layers.conv2d_transpose(fcn_8, filters=L_4_out.get_shape().as_list()[-1],
    kernel_size=4, strides=(2, 2), padding='SAME', name="fcn_9")

    #Connecting FCN_8 to layer 4 for skip connection
    fcn_9_skip = tf.add(fcn_9, L_4_out, name="fcn_9_plus_L_4_out")

    # Upsample
    fcn_10 = tf.layers.conv2d_transpose(fcn_9_skip, filters=L_3_out.get_shape().as_list()[-1],
    kernel_size=4, strides=(2, 2), padding='SAME', name="fcn_10")

    # Add skip connection
    fcn_10_skip = tf.add(fcn_10, L_3_out, name="fcn_10_plus_L_3_out")

    # Upsample again. 8x8 stride, hence the name
    fcn_11 = tf.layers.conv2d_transpose(fcn_10_skip, filters=num_classes,
    kernel_size=16, strides=(8, 8), padding='SAME', name="fcn_11")

    return fcn_11